
sudo su
pip uninstall tensorflow-gpu tensorflow

pip install transformers datasets 'accelerate>=0.26.0' -i https://mirrors.aliyun.com/pypi/simple
pip install numpy -i https://mirrors.aliyun.com/pypi/simple --upgrade


In [9]:
import os

os.environ["HF_ENDPOINT"] = "http://223.109.239.18:20281/#browse/browse:hf"
os.environ["HF_HOME"] = "/home/vipuser/.cache/huggingface"
os.environ["HF_HUB_CACHE"] = "/home/vipuser/.cache/huggingface/hub"
os.environ["TRANSFORMERS_CACHE"] = "/home/vipuser/.cache/huggingface/transformers"
os.makedirs(os.environ["HF_HUB_CACHE"], exist_ok=True)
os.makedirs(os.environ["TRANSFORMERS_CACHE"], exist_ok=True)


# obfuscation

In [4]:
import torch
import sys
import os
from transformers import AutoTokenizer, AutoModelForCausalLM

repo_path = "/home/vipuser/LoRO/LoRO/"  
if os.path.exists(repo_path) and repo_path not in sys.path:
    sys.path.append(repo_path)

try:
    from utils import model_obfuscation
except ImportError as e:
    print('wrong repo_path')
    sys.exit(1)

# ==========================================
# 1. 加载目标模型 (Private Model)
# ==========================================
model_id = "zfdev/squad_v2-16bit-Qwen2.5-1.5B-Instruct"
device = "cpu"
save_path = "/home/vipuser/LoRO/LoRO_attack/loro_qwen_1_5B.pt"

print(f"正在加载模型: {model_id} ...")
# Qwen 需要 trust_remote_code=True
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
# 使用 AutoModelForCausalLM 因为这是一个生成任务 (SQuAD)
# New line: Add force_download=True
model = AutoModelForCausalLM.from_pretrained(
    model_id, 
    trust_remote_code=True, 
    force_download=False  # <--- Forces a fresh download
).to(device)
print("模型加载完成。准备进行 LoRO 混淆...")

# ==========================================
# 2. 执行混淆 (调用仓库代码)
# ==========================================
noise_magnitude = 1

print(f"开始混淆 (Noise Magnitude: {noise_magnitude})...")
obfuscated_model = model_obfuscation(model, device=device, noise_mag=noise_magnitude, r=24)

# ==========================================
# 4. 保存混淆后的 Checkpoint
# ==========================================
print(f"正在保存混淆后的模型至: {save_path} ...")
torch.save(obfuscated_model.state_dict(), save_path)

print(f"Checkpoint Path: {os.path.abspath(save_path)}")


正在加载模型: zfdev/squad_v2-16bit-Qwen2.5-1.5B-Instruct ...


model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

模型加载完成。准备进行 LoRO 混淆...
开始混淆 (Noise Magnitude: 1)...
Obfuscating: model.layers.0.self_attn.q_proj
Obfuscating: model.layers.0.self_attn.k_proj
Obfuscating: model.layers.0.self_attn.v_proj
Obfuscating: model.layers.0.self_attn.o_proj
Obfuscating: model.layers.0.mlp.gate_proj
Obfuscating: model.layers.0.mlp.up_proj
Obfuscating: model.layers.0.mlp.down_proj
Obfuscating: model.layers.1.self_attn.q_proj
Obfuscating: model.layers.1.self_attn.k_proj
Obfuscating: model.layers.1.self_attn.v_proj
Obfuscating: model.layers.1.self_attn.o_proj
Obfuscating: model.layers.1.mlp.gate_proj
Obfuscating: model.layers.1.mlp.up_proj
Obfuscating: model.layers.1.mlp.down_proj
Obfuscating: model.layers.2.self_attn.q_proj
Obfuscating: model.layers.2.self_attn.k_proj
Obfuscating: model.layers.2.self_attn.v_proj
Obfuscating: model.layers.2.self_attn.o_proj
Obfuscating: model.layers.2.mlp.gate_proj
Obfuscating: model.layers.2.mlp.up_proj
Obfuscating: model.layers.2.mlp.down_proj
Obfuscating: model.layers.3.self_att

# cmp base & ft

In [5]:
import torch
from transformers import AutoModelForCausalLM
import pandas as pd

# ==========================================
# 配置
# ==========================================
model_id_ft = "zfdev/squad_v2-16bit-Qwen2.5-1.5B-Instruct"   # Target
model_id_base = "Qwen/Qwen2.5-1.5B-Instruct"    # Prior (Base)
device = "cpu" # 对比只需 CPU 即可，省显存

print(f"Loading Fine-Tuned Model: {model_id_ft}...")
model_ft = AutoModelForCausalLM.from_pretrained(model_id_ft, trust_remote_code=True).to(device)

print(f"Loading Base Model: {model_id_base}...")
model_base = AutoModelForCausalLM.from_pretrained(model_id_base, trust_remote_code=True).to(device)

print("\nStarting Comparison (FT vs. Base)...")
print("-" * 90)
print(f"{'Layer Name':<50} | {'Cos Sim':<10} | {'Delta Norm':<12} | {'Rel Diff (%)':<12}")
print("-" * 90)

results = []
modules_base = dict(model_base.named_modules())

for name, module_ft in model_ft.named_modules():
    if isinstance(module_ft, torch.nn.Linear):
        if name in modules_base:
            module_base = modules_base[name]
            w_ft = module_ft.weight.detach()
            w_base = module_base.weight.detach()
            
            if w_ft.shape != w_base.shape:
                continue
                
            # 1. Cosine Similarity
            cos_sim = torch.nn.functional.cosine_similarity(
                w_ft.flatten(), w_base.flatten(), dim=0
            ).item()
            
            # 2. Delta (FT - Base)
            delta = w_ft - w_base
            norm_delta = torch.norm(delta).item()
            norm_base = torch.norm(w_base).item()
            
            rel_diff = norm_delta / norm_base if norm_base > 0 else 0.0
            
            print(f"{name:<50} | {cos_sim:.6f}   | {norm_delta:.4f}       | {rel_diff*100:.4f}%")
            
            results.append({
                "Layer": name, "Cos_Sim": cos_sim, 
                "Delta_Norm": norm_delta, "Rel_Diff": rel_diff
            })

df = pd.DataFrame(results)
print("-" * 90)
print(f"Avg Cos Sim: {df['Cos_Sim'].mean():.4f} | Avg Delta Norm: {df['Delta_Norm'].mean():.4f}")

Loading Fine-Tuned Model: zfdev/squad_v2-16bit-Qwen2.5-1.5B-Instruct...
Loading Base Model: Qwen/Qwen2.5-1.5B-Instruct...


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]


Starting Comparison (FT vs. Base)...
------------------------------------------------------------------------------------------
Layer Name                                         | Cos Sim    | Delta Norm   | Rel Diff (%)
------------------------------------------------------------------------------------------
model.layers.0.self_attn.q_proj                    | 0.999997   | 0.1955       | 0.2621%
model.layers.0.self_attn.k_proj                    | 0.999997   | 0.0789       | 0.2254%
model.layers.0.self_attn.v_proj                    | 0.999970   | 0.0915       | 0.7792%
model.layers.0.self_attn.o_proj                    | 0.999977   | 0.2367       | 0.6725%
model.layers.0.mlp.gate_proj                       | 0.999986   | 0.6117       | 0.5364%
model.layers.0.mlp.up_proj                         | 0.999980   | 0.5940       | 0.6271%
model.layers.0.mlp.down_proj                       | 0.999997   | 0.2366       | 0.2383%
model.layers.1.self_attn.q_proj                    | 0.999991  

# SVD

In [6]:
import torch
import torch.nn as nn
from transformers import AutoModelForCausalLM, AutoTokenizer
import os
import copy
from tqdm import tqdm

# ==========================================
# 1. 配置
# ==========================================
# 你的 Base 模型 (攻击者先验)
base_model_id = "Qwen/Qwen2.5-1.5B-Instruct"
# 你的 GT 模型 (仅用于验证攻击结果)
target_model_id = "zfdev/squad_v2-16bit-Qwen2.5-1.5B-Instruct"
# 混淆后的 Checkpoint 路径
obfuscated_checkpoint = "/home/vipuser/LoRO/LoRO_attack/loro_qwen_1_5B.pt"
# 恢复模型的保存路径
save_path_recovered = "/home/vipuser/LoRO/LoRO_attack/recovered_qwen_1_5B_model"

device = "cuda" if torch.cuda.is_available() else "cpu"

# === 攻击关键参数 ===
REMOVE_RANK = 24

# ==========================================
# 2. 模型加载
# ==========================================
print(f"1. Loading Base Model (Prior): {base_model_id}...")
# 必须加 trust_remote_code=True 以支持 Qwen
recovered_model = AutoModelForCausalLM.from_pretrained(base_model_id, trust_remote_code=True).to(device)

print(f"2. Loading Obfuscated Checkpoint: {obfuscated_checkpoint}...")
if not os.path.exists(obfuscated_checkpoint):
    raise FileNotFoundError("混淆 Checkpoint 未找到，请先运行 Cell 1 生成。")
obfus_state_dict = torch.load(obfuscated_checkpoint, map_location=device)

print(f"3. Loading Ground Truth (Validation): {target_model_id}...")
gt_model = AutoModelForCausalLM.from_pretrained(target_model_id, trust_remote_code=True).to(device)

# ==========================================
# 3. 执行 SVD 去噪攻击 (SVD Denoising)
# ==========================================
print(f"\nSTARTING RECOVERY (Removing Top-{REMOVE_RANK} Singular Values as Noise)...")
print("="*60)

similarities = []
relative_errors = []

# 遍历 Base 模型的每一层
# 我们需要修改 recovered_model 的权重
all_modules = list(recovered_model.named_modules())
linear_layers = [(n, m) for n, m in all_modules if isinstance(m, nn.Linear)]

progress_bar = tqdm(linear_layers, desc="Recovering")

for name, module in progress_bar:

    if "lm_head" in name:
        print(f"Keeping Base Weights for {name}")
        continue

    # 1. 构造 LoRO 保存的 key
    # 根据 LoroLinear 代码，权重保存在 "obfus_linear.weight
    obfus_key = f"{name}.obfus_linear.weight"
    
    # 检查该层是否被混淆 (如果没在 checkpoint 里找到，说明该层未混淆，保持 Base 原样)
    if obfus_key not in obfus_state_dict:
        continue
        
    # 2. 获取数据
    W_base = module.weight.detach() # [out, in]
    W_obfus = obfus_state_dict[obfus_key].detach() # [out, in]
    
    # 3. 计算 Diff (包含 Fine-tuning 更新 + LoRO 噪声)
    # Diff = (W_base + Delta_FT + Noise) - W_base = Delta_FT + Noise
    print("size of W_obfus: {W_obfus.shape} | size of W_base: {W_base.shape}")
    Diff = W_obfus - W_base
    
    # 4. SVD 分解
    # 使用 float32 保证精度，Diff 形状通常是 [out_features, in_features]
    U, S, Vh = torch.linalg.svd(Diff.float(), full_matrices=False)

    print(f"Layer: {name} | {S[REMOVE_RANK - 2]}, {S[REMOVE_RANK - 1]}, {S[REMOVE_RANK]}, {S[REMOVE_RANK + 1]}")
    
    # --- 攻击核心逻辑 ---
    # 根据 LoroLinear 定义，噪声是 (B@A).T，其秩为 r (REMOVE_RANK)。
    # 且因为 noise_mag 很大，这 r 个奇异值会非常大，位于 S 的最前端。
    # 我们将前 r 个奇异值置零，即移除了 LoRO 噪声。
    S_clean = S.clone()
    S_clean[:REMOVE_RANK] = 0.0 
    
    # 5. 重构纯净的 Delta (即估算的 Delta_FT)
    Delta_Recovered = (U @ torch.diag(S_clean) @ Vh).to(W_base.dtype)
    
    # 6. 恢复权重并写入模型
    # W_rec = W_base + Delta_Recovered
    module.weight.data = W_base + Delta_Recovered
    
    # 7. 恢复 Bias
    # LoroLinear 代码中: self.obfus_linear.bias = original_linear.bias
    # Bias 没有加噪声，直接从 checkpoint 读取覆盖即可
    obfus_bias_key = f"{name}.obfus_linear.bias"
    if obfus_bias_key in obfus_state_dict and module.bias is not None:
        module.bias.data = obfus_state_dict[obfus_bias_key].detach()
    
    # ==========================
    # 验证环节 (对比 GT)
    # ==========================
    if name in dict(gt_model.named_modules()):
        W_gt = dict(gt_model.named_modules())[name].weight.detach()
        Delta_True = W_gt - W_base
        
        # 计算相似度: 我们的恢复结果 vs 真实的微调增量
        # 如果 sim 接近 1.0，说明我们成功剥离了噪声，保留了微调增量
        sim = torch.nn.functional.cosine_similarity(Delta_True.flatten(), Delta_Recovered.flatten(), dim=0).item()
        
        print(f"Layer: {name} | Similarity with GT Delta: {sim:.4f}")
        
        similarities.append(sim)
        # progress_bar.set_postfix({"Avg Sim": f"{sum(similarities)/len(similarities):.4f}", "Last Sim": f"{sim:.4f}"})

# ==========================================
# 4. 结果保存
# ==========================================

print(f"Saving recovered model to {save_path_recovered}...")
recovered_model.save_pretrained(save_path_recovered)
tokenizer = AutoTokenizer.from_pretrained(base_model_id, trust_remote_code=True)
tokenizer.save_pretrained(save_path_recovered)
print("Saved.")

1. Loading Base Model (Prior): Qwen/Qwen2.5-1.5B-Instruct...
2. Loading Obfuscated Checkpoint: /home/vipuser/LoRO/LoRO_attack/loro_qwen_1_5B.pt...
3. Loading Ground Truth (Validation): zfdev/squad_v2-16bit-Qwen2.5-1.5B-Instruct...

STARTING RECOVERY (Removing Top-24 Singular Values as Noise)...


Recovering:   0%|          | 0/197 [00:00<?, ?it/s]

size of W_obfus: {W_obfus.shape} | size of W_base: {W_base.shape}
Layer: model.layers.0.self_attn.q_proj | 110.98078918457031, 105.23827362060547, 0.1035633310675621, 0.07278374582529068


Recovering:   1%|          | 1/197 [00:00<02:40,  1.22it/s]

Layer: model.layers.0.self_attn.q_proj | Similarity with GT Delta: 0.9816
size of W_obfus: {W_obfus.shape} | size of W_base: {W_base.shape}
Layer: model.layers.0.self_attn.k_proj | 37.55204772949219, 35.60569381713867, 0.036296095699071884, 0.030306849628686905
Layer: model.layers.0.self_attn.k_proj | Similarity with GT Delta: 0.9438
size of W_obfus: {W_obfus.shape} | size of W_base: {W_base.shape}
Layer: model.layers.0.self_attn.v_proj | 38.228233337402344, 37.28694152832031, 0.048876505345106125, 0.02879672311246395
Layer: model.layers.0.self_attn.v_proj | Similarity with GT Delta: 0.9415
size of W_obfus: {W_obfus.shape} | size of W_base: {W_base.shape}


...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Saved.


# cmp ft & recovered

In [7]:
import torch
from transformers import AutoModelForCausalLM
import pandas as pd
import os

# ==========================================
# 配置：对比 Ground Truth (Target) 与 恢复模型 (Recovered)
# ==========================================

target_model_id = "zfdev/squad_v2-16bit-Qwen2.5-1.5B-Instruct" 
recovered_model_path = "/home/vipuser/LoRO/LoRO_attack/recovered_qwen_1_5B_model"

device = "cpu" # 对比只需 CPU 即可，省显存

print(f"Loading Target Model (GT): {target_model_id}...")
try:
    model_gt = AutoModelForCausalLM.from_pretrained(target_model_id, trust_remote_code=True).to(device)
except Exception as e:
    print(f"Error loading GT model: {e}")
    # 如果显存不够，可以尝试加 device_map="cpu" 或 torch_dtype=torch.float16

print(f"Loading Recovered Model: {recovered_model_path}...")

try:
    model_recovered = AutoModelForCausalLM.from_pretrained(recovered_model_path, trust_remote_code=True).to(device)
except Exception as e:
    print(f"Error loading Recovered model: {e}")
    raise e

print("\nStarting Comparison (Target GT vs. Recovered)...")
print("*" * 90)
print(f"{'Layer Name':<50} | {'Cos Sim':<10} | {'Diff Norm':<12} | {'Rel Diff (%)':<12}")
print("-" * 90)

results = []
modules_recovered = dict(model_recovered.named_modules())

# 遍历 GT 模型的层
for name, module_gt in model_gt.named_modules():
    if isinstance(module_gt, torch.nn.Linear):
        if name in modules_recovered:
            module_rec = modules_recovered[name]
            
            # 获取权重
            w_gt = module_gt.weight.detach()
            w_rec = module_rec.weight.detach()
            
            # 检查形状
            if w_gt.shape != w_rec.shape:
                continue
                
            # 1. 计算余弦相似度
            cos_sim = torch.nn.functional.cosine_similarity(
                w_gt.flatten(), w_rec.flatten(), dim=0
            ).item()
            
            # 2. 计算差异 (Diff = GT - Recovered)
            # 如果攻击完美，Diff 应该全是 0
            diff = w_gt - w_rec
            norm_diff = torch.norm(diff).item()
            norm_gt = torch.norm(w_gt).item()
            
            # 相对差异
            rel_diff = norm_diff / norm_gt if norm_gt > 0 else 0.0
            
            # 打印结果 (只打印部分层，或者差异较大的层)
            # 如果 Cos Sim < 0.99 或者 Diff 比较大，说明恢复有问题
            print(f"{name} | {cos_sim:.6f}   | {norm_diff:.4f}       | {rel_diff*100:.4f}%")
            
            results.append({
                "Layer": name, 
                "Cos_Sim": cos_sim, 
                "Diff_Norm": norm_diff, 
                "Rel_Diff": rel_diff
            })

df = pd.DataFrame(results)
print("-" * 90)
print(f"Summary (Target vs Recovered):")
print(f"Avg Cosine Similarity: {df['Cos_Sim'].mean():.6f} (Target: 1.0)")
print(f"Avg Relative Diff:     {df['Rel_Diff'].mean()*100:.6f}% (Target: 0.0%)")
print("-" * 90)


Loading Target Model (GT): zfdev/squad_v2-16bit-Qwen2.5-1.5B-Instruct...
Loading Recovered Model: /home/vipuser/LoRO/LoRO_attack/recovered_qwen_1_5B_model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


Starting Comparison (Target GT vs. Recovered)...
******************************************************************************************
Layer Name                                         | Cos Sim    | Diff Norm    | Rel Diff (%)
------------------------------------------------------------------------------------------
model.layers.0.self_attn.q_proj | 1.000000   | 0.0373       | 0.0500%
model.layers.0.self_attn.k_proj | 1.000000   | 0.0261       | 0.0745%
model.layers.0.self_attn.v_proj | 0.999997   | 0.0308       | 0.2626%
model.layers.0.self_attn.o_proj | 0.999999   | 0.0439       | 0.1248%
model.layers.0.mlp.gate_proj | 1.000000   | 0.0794       | 0.0696%
model.layers.0.mlp.up_proj | 1.000000   | 0.0781       | 0.0824%
model.layers.0.mlp.down_proj | 1.000000   | 0.0347       | 0.0349%
model.layers.1.self_attn.q_proj | 1.000000   | 0.0405       | 0.0793%
model.layers.1.self_attn.k_proj | 0.999999   | 0.0309       | 0.1111%
model.layers.1.self_attn.v_proj | 0.999993   | 0.0473  

In [ ]:
# full-FT with Knockoff (skipped)
# BUT I think we need LoRA due to its mathematical property
# the delta we lost is truncated SVD values (top 24)
# then test accuracy

# test

In [1]:
import os

# os.environ["HF_ENDPOINT"] = "https://hf-mirror.com/"
os.environ["HF_HOME"] = "/home/vipuser/.cache/huggingface"
os.environ["HF_HUB_CACHE"] = "/home/vipuser/.cache/huggingface/hub"
os.environ["TRANSFORMERS_CACHE"] = "/home/vipuser/.cache/huggingface/transformers"
os.makedirs(os.environ["HF_HUB_CACHE"], exist_ok=True)
os.makedirs(os.environ["TRANSFORMERS_CACHE"], exist_ok=True)


## recovered

In [3]:
# rec
import torch
import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset

# ==========================================
# 1. 类似 notebook 中的加载部分
# ==========================================
# 你的模型路径
model_path = "/home/vipuser/LoRO/LoRO_attack/recovered_qwen_1_5B_model"
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Loading model from {model_path}...")

try:
    tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        torch_dtype=torch.bfloat16,
        trust_remote_code=True,
        device_map=device
    ).eval()
except Exception as e:
    print(f"Error: {e}")
    # 如果加载本地失败，演示用可以注释掉上面，打开下面这行
    # raise e

# ==========================================
# 2. 加载数据 (对应 notebook cell 8)
# ==========================================
print("Loading SQuAD v2 dataset...")
# 注意：你现在的任务是 SQuAD v2，比 v1 多了无解的情况
dataset = load_dataset("squad_v2", split="validation")

# dataset = dataset.select(range(100)) # 调试用，跑全量时注释掉

print(f"Start evaluating on {len(dataset)} samples...")

# ==========================================
# 3. 核心评估循环 (复刻 notebook cell 9)
# ==========================================
correct = 0
total = 0

# 进度条
progress_bar = tqdm.tqdm(range(len(dataset)))

for i in progress_bar:
    # 1. 获取数据
    item = dataset[i]
    context = item['context']
    question = item['question']
    gold_answers = item['answers']['text'] # 标准答案列表

    # 2. 构建 Prompt (因为是生成模型，不能用 QA pipeline，要用 Prompt 模拟)
    # 格式严格对应你微调时的格式
    prompt = f"Context: {context}\nQuestion: {question}\nAnswer:"

    messages = [
        # 如果微调时有 system prompt 请加上，没有就删掉这行
        {"role": "system", "content": "Answer the question exactly based on the context. If unanswerable, say 'unanswerable'."},
        {"role": "user", "content": prompt}
    ]

    # 3. 模型推理 (代替 pipeline)
    input_text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(input_text, return_tensors="pt").to(device)

    try:
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=50, # 答案通常很短
                do_sample=False,   # 贪婪搜索，保证结果确定性
                pad_token_id=tokenizer.eos_token_id
            )

        # 提取生成的答案
        generated_answer = tokenizer.decode(outputs[0][len(inputs.input_ids[0]):], skip_special_tokens=True).strip()

    except Exception as e:
        print(f"Error at index {i}: {e}")
        generated_answer = ""

    # 4. 判分逻辑 (完全复刻 notebook 的 exact match 逻辑)
    # 逻辑：只要生成的答案 在 标准答案列表里，就算对

    is_correct = False

    # 清洗一下字符串（忽略大小写和首尾空格）
    pred_clean = generated_answer.lower().strip()

    if len(gold_answers) > 0:
        # 情况 A: 有标准答案 (和 SQuAD v1 一样)
        # notebook 里的逻辑: if answer == result['answer']
        for answer in gold_answers:
            if answer.lower().strip() == pred_clean: # 只要命中一个就算对
                is_correct = True
                break
    else:
        # 情况 B: 无解 (SQuAD v2 特有)
        # 假设微调时教过模型输出 'unanswerable'
        if "unanswerable" in pred_clean:
            is_correct = True

    if is_correct:
        correct += 1

    total += 1

    # 实时更新显示 acc
    progress_bar.set_postfix({'acc': f"{correct/total:.2%}"})

# ==========================================
# 4. 打印最终结果
# ==========================================
print("\n" + "="*30)
print("correct:{}, total:{}, accuracy:{}".format(correct, total, correct/total))
print("="*30)

Loading model from /home/vipuser/LoRO/LoRO_attack/recovered_qwen_1_5B_model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading SQuAD v2 dataset...


Using the latest cached version of the dataset since squad_v2 couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'squad_v2' at /home/vipuser/.cache/huggingface/datasets/squad_v2/squad_v2/0.0.0/3ffb306f725f7d2ce8394bc1873b24868140c412 (last modified on Thu Dec 25 18:00:45 2025).


Start evaluating on 11873 samples...


100%|██████████| 11873/11873 [45:38<00:00,  4.34it/s, acc=36.94%] 


correct:4386, total:11873, accuracy:0.3694095847721722


## final

## base

In [ ]:
# base
import torch
import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset

# ==========================================
# 1. 类似 notebook 中的加载部分
# ==========================================
# 你的模型路径
model_path = "Qwen/Qwen2.5-1.5B-Instruct"
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Loading model from {model_path}...")

try:
    tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        torch_dtype=torch.bfloat16,
        trust_remote_code=True,
        device_map=device
    ).eval()
except Exception as e:
    print(f"Error: {e}")
    # 如果加载本地失败，演示用可以注释掉上面，打开下面这行
    # raise e

# ==========================================
# 2. 加载数据 (对应 notebook cell 8)
# ==========================================
print("Loading SQuAD v2 dataset...")
# 注意：你现在的任务是 SQuAD v2，比 v1 多了无解的情况
dataset = load_dataset("squad_v2", split="validation")

# dataset = dataset.select(range(100)) # 调试用，跑全量时注释掉

print(f"Start evaluating on {len(dataset)} samples...")

# ==========================================
# 3. 核心评估循环 (复刻 notebook cell 9)
# ==========================================
correct = 0
total = 0

# 进度条
progress_bar = tqdm.tqdm(range(len(dataset)))

for i in progress_bar:
    # 1. 获取数据
    item = dataset[i]
    context = item['context']
    question = item['question']
    gold_answers = item['answers']['text'] # 标准答案列表

    # 2. 构建 Prompt (因为是生成模型，不能用 QA pipeline，要用 Prompt 模拟)
    # 格式严格对应你微调时的格式
    prompt = f"Context: {context}\nQuestion: {question}\nAnswer:"

    messages = [
        # 如果微调时有 system prompt 请加上，没有就删掉这行
        {"role": "system", "content": "Answer the question exactly based on the context. If unanswerable, say 'unanswerable'."},
        {"role": "user", "content": prompt}
    ]

    # 3. 模型推理 (代替 pipeline)
    input_text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(input_text, return_tensors="pt").to(device)

    try:
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=50, # 答案通常很短
                do_sample=False,   # 贪婪搜索，保证结果确定性
                pad_token_id=tokenizer.eos_token_id
            )

        # 提取生成的答案
        generated_answer = tokenizer.decode(outputs[0][len(inputs.input_ids[0]):], skip_special_tokens=True).strip()

    except Exception as e:
        print(f"Error at index {i}: {e}")
        generated_answer = ""

    # 4. 判分逻辑 (完全复刻 notebook 的 exact match 逻辑)
    # 逻辑：只要生成的答案 在 标准答案列表里，就算对

    is_correct = False

    # 清洗一下字符串（忽略大小写和首尾空格）
    pred_clean = generated_answer.lower().strip()

    if len(gold_answers) > 0:
        # 情况 A: 有标准答案 (和 SQuAD v1 一样)
        # notebook 里的逻辑: if answer == result['answer']
        for answer in gold_answers:
            if answer.lower().strip() == pred_clean: # 只要命中一个就算对
                is_correct = True
                break
    else:
        # 情况 B: 无解 (SQuAD v2 特有)
        # 假设微调时教过模型输出 'unanswerable'
        if "unanswerable" in pred_clean:
            is_correct = True

    if is_correct:
        correct += 1

    total += 1

    # 实时更新显示 acc
    progress_bar.set_postfix({'acc': f"{correct/total:.2%}"})

# ==========================================
# 4. 打印最终结果
# ==========================================
print("\n" + "="*30)
print("correct:{}, total:{}, accuracy:{}".format(correct, total, correct/total))
print("="*30)

Loading model from Qwen/Qwen2.5-1.5B-Instruct...
Loading SQuAD v2 dataset...


Using the latest cached version of the dataset since squad_v2 couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'squad_v2' at /home/vipuser/.cache/huggingface/datasets/squad_v2/squad_v2/0.0.0/3ffb306f725f7d2ce8394bc1873b24868140c412 (last modified on Thu Dec 25 18:00:45 2025).


Start evaluating on 11873 samples...


100%|██████████| 11873/11873 [3:19:14<00:00,  1.01s/it, acc=17.96%] 


correct:2132, total:11873, accuracy:0.17956708498273394


## ft

In [5]:
# ft
import torch
import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset

# ==========================================
# 1. 类似 notebook 中的加载部分
# ==========================================
# 你的模型路径
model_path = "zfdev/squad_v2-16bit-Qwen2.5-1.5B-Instruct"
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Loading model from {model_path}...")

try:
    tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        torch_dtype=torch.bfloat16,
        trust_remote_code=True,
        device_map=device
    ).eval()
except Exception as e:
    print(f"Error: {e}")
    # 如果加载本地失败，演示用可以注释掉上面，打开下面这行
    # raise e

# ==========================================
# 2. 加载数据 (对应 notebook cell 8)
# ==========================================
print("Loading SQuAD v2 dataset...")
# 注意：你现在的任务是 SQuAD v2，比 v1 多了无解的情况
dataset = load_dataset("squad_v2", split="validation")

# dataset = dataset.select(range(100)) # 调试用，跑全量时注释掉

print(f"Start evaluating on {len(dataset)} samples...")

# ==========================================
# 3. 核心评估循环 (复刻 notebook cell 9)
# ==========================================
correct = 0
total = 0

# 进度条
progress_bar = tqdm.tqdm(range(len(dataset)))

for i in progress_bar:
    # 1. 获取数据
    item = dataset[i]
    context = item['context']
    question = item['question']
    gold_answers = item['answers']['text'] # 标准答案列表

    # 2. 构建 Prompt (因为是生成模型，不能用 QA pipeline，要用 Prompt 模拟)
    # 格式严格对应你微调时的格式
    prompt = f"Context: {context}\nQuestion: {question}\nAnswer:"

    messages = [
        # 如果微调时有 system prompt 请加上，没有就删掉这行
        {"role": "system", "content": "Answer the question exactly based on the context. If unanswerable, say 'unanswerable'."},
        {"role": "user", "content": prompt}
    ]

    # 3. 模型推理 (代替 pipeline)
    input_text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(input_text, return_tensors="pt").to(device)

    try:
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=50, # 答案通常很短
                do_sample=False,   # 贪婪搜索，保证结果确定性
                pad_token_id=tokenizer.eos_token_id
            )

        # 提取生成的答案
        generated_answer = tokenizer.decode(outputs[0][len(inputs.input_ids[0]):], skip_special_tokens=True).strip()

    except Exception as e:
        print(f"Error at index {i}: {e}")
        generated_answer = ""

    # 4. 判分逻辑 (完全复刻 notebook 的 exact match 逻辑)
    # 逻辑：只要生成的答案 在 标准答案列表里，就算对

    is_correct = False

    # 清洗一下字符串（忽略大小写和首尾空格）
    pred_clean = generated_answer.lower().strip()

    if len(gold_answers) > 0:
        # 情况 A: 有标准答案 (和 SQuAD v1 一样)
        # notebook 里的逻辑: if answer == result['answer']
        for answer in gold_answers:
            if answer.lower().strip() == pred_clean: # 只要命中一个就算对
                is_correct = True
                break
    else:
        # 情况 B: 无解 (SQuAD v2 特有)
        # 假设微调时教过模型输出 'unanswerable'
        if "unanswerable" in pred_clean:
            is_correct = True

    if is_correct:
        correct += 1

    total += 1

    # 实时更新显示 acc
    progress_bar.set_postfix({'acc': f"{correct/total:.2%}"})

# ==========================================
# 4. 打印最终结果
# ==========================================
print("\n" + "="*30)
print("correct:{}, total:{}, accuracy:{}".format(correct, total, correct/total))
print("="*30)

Loading model from zfdev/squad_v2-16bit-Qwen2.5-1.5B-Instruct...
Loading SQuAD v2 dataset...


Using the latest cached version of the dataset since squad_v2 couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'squad_v2' at /home/vipuser/.cache/huggingface/datasets/squad_v2/squad_v2/0.0.0/3ffb306f725f7d2ce8394bc1873b24868140c412 (last modified on Thu Dec 25 18:00:45 2025).


Start evaluating on 11873 samples...


100%|██████████| 11873/11873 [37:45<00:00,  5.24it/s, acc=37.08%] 


correct:4403, total:11873, accuracy:0.37084140486818834
